# Noise Detection Using Machine Learning

## Authors: 


***Lucas Li, Kailin Ke, Ryan Zhang*** 

California Polytechnic State University, San Luis Obispo;

Computer Science & Software Engineering Department

## Abstract

Modern life is filled with noises, but with the abundance of noise, people find it hard to identify the source for the noises, and the automation of detecting and recognizing noises is also a field that has great usage. Our group project, Noise detection AI is a binary classifier that processes audio files and predicts whether the sound is a noise. We will use labeled audio to train the model, and it will consequently produce predictions. 

Possible users for our product can be noise-cancel headphone companies, apartment management agencies, and all those individuals or organizations that require or are trying to establish a quiet environment. With our technology, it would be possible to recognize different kinds of noises so they can do further research or apply our findings. 

We will use librosa to transform the audio files into python arrays, and use sequential, knn, cnn models to train with the data. The dataset comes from https://github.com/karolpiczak/ESC-50.git and all audios are labeled. 


# Copyright: Apache 2.0 License



```
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

```


# Colab Basics




## Colab and Jupyter Notebooks


# Overview

This is an audio classification AI model building module. Data accessing, processing, and training are the three major parts of this lab. Major Model used in this lab is sequential model. 

## Prerequisites

Readers should have basic knowledge on audio processing, MFCC formatting, and neural networks. 

Required libraries: librosa, tensorflow

In [3]:
import librosa
import librosa.display

##Learning Objectives

*   Learn how sequential model is built and tested
*   Learn how to load and process audio 
*   Learn how to separate training data and testing data



# Data Set


Load dataset from Google cloud drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


You can download and upload the folder to google drive from  https://github.com/karolpiczak/ESC-50.git, make it sure that you rename the folder as 'audio' before uploading. 

In [5]:
import os
path = '/content/drive/MyDrive/audio/'
audio_list = os.listdir(path)

## Data Preparation

### Preprocessing

Feature extractor will take file name and process it by mfcc transformation and returns a python array

In [6]:
import numpy as np

def features_extractor(file):
    audio, sample_rate = librosa.load(file, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

Now we iterate through every audio file and extract features using MFCC

In [8]:
extracted_features=[]
i = 0
print('.'*100)
for filename in audio_list:
  audio_path = path + filename
  class_label = 1 if filename[0] == '5' else 0
  data = features_extractor(audio_path)
  extracted_features.append([data, class_label])
  i += 1
  if i % 20 == 0:
    print('.', end='')

....................................................................................................
....................................................................................................

put data into dataframe using Pandas

In [9]:
import pandas as pd
# mfcc feature dataframe construction
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-581.5875, 20.211653, -7.788693, -0.61071056,...",0
1,"[-527.4214, 31.830875, -1.0057539, 41.409687, ...",0
2,"[-135.3644, 97.22801, -77.85827, 11.474359, -1...",0
3,"[-315.32336, 39.60734, -47.233456, 28.209513, ...",0
4,"[-429.4027, 97.49279, -36.4146, 13.635209, 48....",0


catagorize the label and load the data to inputs and outputs

In [10]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

Separate the data into training data and testing data

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

# Model 

## Model Selection

* *italicized text* Sequential Model

## Model Design

A sequential model with multiple layers will be built 

build the model

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [13]:
### No of classes
num_labels=y.shape[1]
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40, )))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4100      
                                                                 
 activation (Activation)     (None, 100)               0         
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               20200     
                                                                 
 activation_1 (Activation)   (None, 200)               0         
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               2

In [15]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

## Model Training

Train the model with processed data

In [16]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100
43/50 [========================>.....] - ETA: 0s - loss: 10.0053 - accuracy: 0.6831
Epoch 1: val_loss improved from inf to 0.93699, saving model to saved_models/audio_classification.hdf5
50/50 [==============================] - 2s 11ms/step - loss: 9.4126 - accuracy: 0.6812 - val_loss: 0.9370 - val_accuracy: 0.8150
Epoch 2/100
46/50 [==========================>...] - ETA: 0s - loss: 4.1260 - accuracy: 0.6834
Epoch 2: val_loss improved from 0.93699 to 0.73480, saving model to saved_models/audio_classification.hdf5
50/50 [==============================] - 0s 6ms/step - loss: 4.0493 - accuracy: 0.6825 - val_loss: 0.7348 - val_accuracy: 0.8150
Epoch 3/100
44/50 [=========================>....] - ETA: 0s - loss: 2.5720 - accuracy: 0.6712
Epoch 3: val_loss improved from 0.73480 to 0.52577, saving model to saved_models/audio_classification.hdf5
50/50 [==============================] - 0s 5ms/step - loss: 2.5012 - accuracy: 0.6712 - val_loss: 0.5258 - val_accuracy: 0.8100
Epoch 4/1

## Model Validation

print test accuracy

In [17]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy)

[0.4875955879688263, 0.8149999976158142]


# Conclusions

Accuracy still needs improvement, and we will try different models and data processsing methods to have a better or more complete result. 

Based on our current knowledge level and information, we are not certain about if a different approach will result in a better outcome. However, we'll try more models

#Future Work

Build and test more models, also try to process the audios

# Acknowledgements





dataset cite: https://github.com/karolpiczak/ESC-50.git
